# Demographic Data Manipulation Notebook

In [5]:
import pandas as pd
import numpy as np
from pathlib import Path

# Define base path
base_path = Path("./app_data")
# base_path = Path("data/visualization/streamlit/app_data")

# Utility function to clean numeric columns with spaces and commas
def clean_numeric(value):
    if pd.isna(value):
        return np.nan
    if isinstance(value, str):
        # Remove spaces and commas
        value = value.replace(' ', '').replace(',', '')
    try:
        return float(value)
    except:
        return np.nan

# Utility function to fix district name encoding issues
def fix_district_name(name):
    if pd.isna(name):
        return name
    # Fix common encoding issues
    name = name.replace('Bao�an', "Bao'an")
    name = name.replace('Baoan', "Bao'an")
    return name

print("Loading and processing datasets...")
print("=" * 60)

Loading and processing datasets...


## Part 1: General Demographics and Indicators

This section loads and processes the general demographic files into a single condensed dataset.

In [6]:
# ============================================================================
# PART 1: GENERAL DEMOGRAPHICS AND INDICATORS
# ============================================================================

# 1. Load Housing Prices
print("\n1. Loading housing prices...")
housing = pd.read_csv(base_path / "avg_housing_price_per_sq_meter_by_district.csv")
# Clean numeric values (remove commas)
for col in housing.columns[1:]:
    housing[col] = housing[col].apply(clean_numeric)
print(f"   Shape: {housing.shape}")
print(f"   Columns: {list(housing.columns)}")

# 2. Load GDP
print("\n2. Loading GDP data...")
gdp = pd.read_csv(base_path / "GDP_by_district_10k_yuan.csv")
gdp['District'] = gdp['District'].apply(fix_district_name)
print(f"   Shape: {gdp.shape}")
print(f"   Districts: {gdp['District'].tolist()}")

# 3. Load Household Population with Gender
print("\n3. Loading household population data...")
household = pd.read_csv(base_path / "household_population_with_gender_by_district.csv")
household['District'] = household['District'].apply(fix_district_name)
# Clean numeric columns
for col in household.columns[1:]:
    household[col] = household[col].apply(clean_numeric)
print(f"   Shape: {household.shape}")
print(f"   Columns: {list(household.columns)}")

# 4. Load Employed Workers (Total only)
print("\n4. Loading employed workers data (Total column only)...")
employed = pd.read_csv(base_path / "number_of_fully_employed_workers_at_year_end_by_district.csv")
employed = employed[['District', 'Total']].copy()
employed['District'] = employed['District'].apply(fix_district_name)
employed['Total'] = employed['Total'].apply(clean_numeric)
employed.rename(columns={'Total': 'Total_Employed_Workers'}, inplace=True)
print(f"   Shape: {employed.shape}")
print(f"   Sample values: {employed.head(3).to_dict('records')}")

# 5. Load Land Area and Population Density
print("\n5. Loading land area and population density...")
land_pop = pd.read_csv(base_path / "total_land_area_population_and_density_by_district.csv")
# Select specific columns
land_pop = land_pop[['Region', 'Land Area (km)', 'Year-end Permanent Population (10,000 persons)', 
                      'Population Density (persons/km)']].copy()
land_pop.rename(columns={'Region': 'District'}, inplace=True)
land_pop['District'] = land_pop['District'].apply(fix_district_name)
# Clean numeric columns
for col in land_pop.columns[1:]:
    land_pop[col] = land_pop[col].apply(clean_numeric)
print(f"   Shape: {land_pop.shape}")
print(f"   Columns: {list(land_pop.columns)}")


1. Loading housing prices...
   Shape: (3, 7)
   Columns: ['Month-Year', 'Luohu', 'Futian', 'Nanshan', 'Yantian', "Bao'an", 'Longgang']

2. Loading GDP data...
   Shape: (13, 2)
   Districts: ['Total', 'Futian', 'Luohu', 'Yantian', 'Nanshan', "Original_Bao'an", "Bao'an", 'Longhua', 'Guangming', 'Original_Longgang', 'Longgang', 'Pingshan', 'Dapeng']

3. Loading household population data...
   Shape: (10, 5)
   Columns: ['District', 'Household Registration Population (10,000 persons)', 'Men (10,000 persons)', 'Women (10,000 persons)', 'Sex Ratio (?=100)']

4. Loading employed workers data (Total column only)...
   Shape: (10, 2)
   Sample values: [{'District': 'Futian', 'Total_Employed_Workers': 977584.0}, {'District': 'Luohu', 'Total_Employed_Workers': 394900.0}, {'District': 'Yantian', 'Total_Employed_Workers': 65854.0}]

5. Loading land area and population density...
   Shape: (11, 4)
   Columns: ['District', 'Land Area (km)', 'Year-end Permanent Population (10,000 persons)', 'Popula

In [7]:
# Merge all general demographics datasets
print("\n" + "=" * 60)
print("MERGING GENERAL DEMOGRAPHICS...")
print("=" * 60)

# Start with GDP as base (has all districts including historical ones)
general_demo = gdp.copy()

# Merge household population
general_demo = general_demo.merge(household, on='District', how='left')

# Merge employed workers
general_demo = general_demo.merge(employed, on='District', how='left')

# Merge land area and population
general_demo = general_demo.merge(land_pop, on='District', how='left')

# Note: Housing prices are time-series (monthly) with different structure
# Keep them separate for now and document the relationship
print(f"\nGeneral Demographics Shape: {general_demo.shape}")
print(f"Columns: {list(general_demo.columns)}")
print(f"\nFirst few rows:")
print(general_demo.head())

# Save general demographics
output_file_1 = "condensed_general_demographics.csv"
general_demo.to_csv(output_file_1, index=False)
print(f"\n✓ Saved to: {output_file_1}")

# Save housing prices separately (time series)
output_file_housing = "condensed_housing_prices_timeseries.csv"
housing.to_csv(output_file_housing, index=False)
print(f"✓ Saved housing prices to: {output_file_housing}")
print(f"  (Note: Housing prices are monthly time-series data, kept separate)")


MERGING GENERAL DEMOGRAPHICS...

General Demographics Shape: (13, 10)
Columns: ['District', 'GDP', 'Household Registration Population (10,000 persons)', 'Men (10,000 persons)', 'Women (10,000 persons)', 'Sex Ratio (?=100)', 'Total_Employed_Workers', 'Land Area (km)', 'Year-end Permanent Population (10,000 persons)', 'Population Density (persons/km)']

First few rows:
  District        GDP  Household Registration Population (10,000 persons)  \
0    Total  194926012                                                NaN    
1   Futian   35572870                                              98.97    
2    Luohu   19724939                                              61.19    
3  Yantian    5375327                                               7.20    
4  Nanshan   38452711                                              85.79    

   Men (10,000 persons)  Women (10,000 persons)  Sex Ratio (?=100)  \
0                   NaN                     NaN                NaN   
1                 49.40   

## Part 2: Detailed Economic and Financial Indicators

This section loads and processes the economic/financial files into a single condensed dataset.

In [ ]:
# ============================================================================
# PART 2: DETAILED ECONOMIC AND FINANCIAL INDICATORS
# ============================================================================

print("\n" + "=" * 60)
print("LOADING ECONOMIC/FINANCIAL DATASETS...")
print("=" * 60)

# 1. Load Economy Sector Proportions
print("\n1. Loading economy sector proportions...")
economy_sectors = pd.read_csv(base_path / "economy_sector_proportions_by_district_by_unit.csv")
economy_sectors['District'] = economy_sectors['District'].apply(fix_district_name)
# Clean all numeric columns
for col in economy_sectors.columns[1:]:
    economy_sectors[col] = economy_sectors[col].apply(clean_numeric)
print(f"   Shape: {economy_sectors.shape}")
print(f"   Columns: {list(economy_sectors.columns)}")

# 2. Load Enterprise Financial Indicators
print("\n2. Loading enterprise financial indicators...")
enterprise = pd.read_csv(base_path / "enterprise_financial_indicators_by_district.csv")
enterprise.rename(columns={'Region': 'District'}, inplace=True)
enterprise['District'] = enterprise['District'].apply(fix_district_name)
# Clean all numeric columns
for col in enterprise.columns[1:]:
    enterprise[col] = enterprise[col].apply(clean_numeric)
print(f"   Shape: {enterprise.shape}")
print(f"   Columns: {list(enterprise.columns)}")

# 3. Load Full Employment by Sector (all columns this time)
print("\n3. Loading fully employed workers by sector (all columns)...")
employed_full = pd.read_csv(base_path / "number_of_fully_employed_workers_at_year_end_by_district.csv")
employed_full['District'] = employed_full['District'].apply(fix_district_name)
# Clean all numeric columns
for col in employed_full.columns[1:]:
    employed_full[col] = employed_full[col].apply(clean_numeric)
print(f"   Shape: {employed_full.shape}")
print(f"   Columns: {list(employed_full.columns)}")
print(f"   Number of sector columns: {len(employed_full.columns) - 1}")

In [ ]:
# Merge economic/financial datasets
print("\n" + "=" * 60)
print("MERGING ECONOMIC/FINANCIAL INDICATORS...")
print("=" * 60)

# Start with economy sectors
economic_indicators = economy_sectors.copy()

# Merge enterprise financial indicators
economic_indicators = economic_indicators.merge(
    enterprise, 
    on='District', 
    how='outer',
    suffixes=('_sectors', '_enterprise')
)

# Merge full employment by sector
economic_indicators = economic_indicators.merge(
    employed_full,
    on='District',
    how='outer',
    suffixes=('', '_employment')
)

print(f"\nEconomic Indicators Shape: {economic_indicators.shape}")
print(f"Total Columns: {len(economic_indicators.columns)}")
print(f"Districts: {economic_indicators['District'].tolist()}")

# Save economic indicators
output_file_2 = "condensed_economic_indicators.csv"
economic_indicators.to_csv(output_file_2, index=False)
print(f"\n✓ Saved to: {output_file_2}")

## Summary

Review the created files and their contents.

In [ ]:
# ============================================================================
# SUMMARY
# ============================================================================

print("\n" + "=" * 60)
print("SUMMARY OF CONDENSED DATASETS")
print("=" * 60)

print("\n1. GENERAL DEMOGRAPHICS (condensed_general_demographics.csv)")
print(f"   - Rows: {len(general_demo)}")
print(f"   - Columns: {len(general_demo.columns)}")
print(f"   - File size: {Path(output_file_1).stat().st_size / 1024:.2f} KB")
print(f"   - Districts included: {len(general_demo)}")
print("\n   Column breakdown:")
for col in general_demo.columns:
    null_count = general_demo[col].isna().sum()
    print(f"     • {col}: {null_count} nulls")

print("\n2. HOUSING PRICES TIME SERIES (condensed_housing_prices_timeseries.csv)")
print(f"   - Rows: {len(housing)}")
print(f"   - Columns: {len(housing.columns)}")
print(f"   - File size: {Path(output_file_housing).stat().st_size / 1024:.2f} KB")
print(f"   - Time periods: {len(housing)}")
print(f"   - Districts with data: {', '.join([col for col in housing.columns[1:]])}")

print("\n3. ECONOMIC INDICATORS (condensed_economic_indicators.csv)")
print(f"   - Rows: {len(economic_indicators)}")
print(f"   - Columns: {len(economic_indicators.columns)}")
print(f"   - File size: {Path(output_file_2).stat().st_size / 1024:.2f} KB")
print(f"   - Districts included: {len(economic_indicators)}")
print("\n   First 10 columns:")
for col in economic_indicators.columns[:10]:
    null_count = economic_indicators[col].isna().sum()
    print(f"     • {col}: {null_count} nulls")
print(f"   ... and {len(economic_indicators.columns) - 10} more columns")

print("\n" + "=" * 60)
print("✓ PROCESSING COMPLETE!")
print("=" * 60)
print("\nOutput files created:")
print(f"  1. {output_file_1}")
print(f"  2. {output_file_housing}")
print(f"  3. {output_file_2}")